In [1]:
import duckdb
import pandas as pd

##TODO: Falta bajar el dataset directamente de Kaggle
# import kagglehub
# from kagglehub import KaggleDatasetAdapter

# # Load the dataset from KaggleHub
# df = kagglehub.load_dataset(
#     KaggleDatasetAdapter.PANDAS,
#     "alejandroczernikier/properati-argentina-dataset",
#     file_path="/data/raw"
# )


con = duckdb.connect("app/data/DB/entrenamiento.duckdb")

# guardar CSV como Parquet
con.execute(
    """
COPY (
    SELECT * FROM read_csv_auto('app/data/RAW/entrenamiento.csv')
) TO 'app/data/parquet/entrenamiento.parquet' (FORMAT PARQUET)
"""
)


# Crear tabla en duckdb db desde el archivo csv con datos en crudo

con.execute(
    "CREATE TABLE  IF NOT EXISTS datos_raw AS SELECT * FROM read_csv_auto('app/data/RAW/entrenamiento.csv')"
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [2]:
tables = con.execute("SHOW TABLES").fetchall()
print(tables)

[('datos_clean',), ('datos_raw',), ('datosraw_',)]


In [3]:
# cant. de filas
n_filas = duckdb.query(
    "SELECT COUNT(*) FROM 'app/data/parquet/entrenamiento.parquet'"
).fetchone()[0]
print("Filas:", n_filas)

Filas: 992192


In [4]:
# profiling del dataset en parquet
parquet_file = "app/data/parquet/entrenamiento.parquet"

con = duckdb.connect()

# Obtener metadatos de columnas
cols = con.execute(
    f"""
DESCRIBE SELECT * FROM parquet_scan('{parquet_file}')
"""
).fetchdf()

profiling_results = []

for _, row in cols.iterrows():
    col = row["column_name"]
    dtype = row["column_type"].lower()

    is_numeric = any(t in dtype for t in ["int", "decimal", "double", "float", "real"])

    stats_query = f"""
    SELECT
        '{col}' AS column,
        '{dtype}' AS dtype,
        COUNT(*) AS total,
        COUNT(*) FILTER (WHERE "{col}" IS NULL) AS nulls,
        COUNT(DISTINCT "{col}") AS distinct_values,
        { "MIN(" + col + ")" if is_numeric else "NULL" } AS min,
        { "MAX(" + col + ")" if is_numeric else "NULL" } AS max,
        { "AVG(" + col + ")" if is_numeric else "NULL" } AS mean,
        { "MEDIAN(" + col + ")" if is_numeric else "NULL" } AS median,
        { "QUANTILE(" + col + ", 0.25)" if is_numeric else "NULL" } AS Q1,
        { "QUANTILE(" + col + ", 0.75)" if is_numeric else "NULL" } AS Q3
    FROM parquet_scan('{parquet_file}')
    """

    stats = con.execute(stats_query).fetchdf()
    profiling_results.append(stats)

profile_df = pd.concat(profiling_results)

print("profiling completado")
display(profile_df)

profiling completado


C:\Users\beln3\AppData\Local\Temp\ipykernel_33196\2466087151.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  profile_df = pd.concat(profiling_results)


,column,dtype,total,nulls,distinct_values,min,max,mean,median,Q1,Q3
0,id,bigint,992192,0,992192,1.000000,1.000000e+06,500338.438758,500641.500000,250259.000000,750507.000000
0,ad_type,varchar,992192,0,1,NaN,NaN,NaN,NaN,NaN,NaN
0,start_date,date,992192,0,346,NaN,NaN,NaN,NaN,NaN,NaN
0,end_date,date,992192,0,450,NaN,NaN,NaN,NaN,NaN,NaN
0,created_on,date,992192,0,346,NaN,NaN,NaN,NaN,NaN,NaN
0,lat,double,992192,153198,317046,-180.000000,-3.504503e+01,-59.266586,-58.444383,-58.756011,-58.058966
0,lon,double,992192,152263,315745,-54.842464,8.505113e+01,-34.793745,-34.610508,-34.921709,-34.459156
0,l1,varchar,992192,0,4,NaN,NaN,NaN,NaN,NaN,NaN
0,l2,varchar,992192,0,42,NaN,NaN,NaN,NaN,NaN,NaN
0,l3,varchar,992192,57455,1262,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = duckdb.query(
    """
    SELECT *
    FROM 'app/data/parquet/entrenamiento.parquet'
    WHERE l2 = 'Capital Federal'
"""
).df()

In [6]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249738 entries, 0 to 249737
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   id               249738 non-null  int64         
 1   ad_type          249738 non-null  object        
 2   start_date       249738 non-null  datetime64[us]
 3   end_date         249738 non-null  datetime64[us]
 4   created_on       249738 non-null  datetime64[us]
 5   lat              217583 non-null  float64       
 6   lon              217705 non-null  float64       
 7   l1               249738 non-null  object        
 8   l2               249738 non-null  object        
 9   l3               225456 non-null  object        
 10  l4               10091 non-null   object        
 11  l5               0 non-null       object        
 12  l6               0 non-null       object        
 13  rooms            171449 non-null  float64       
 14  bedrooms         122

In [7]:
# profile del df solo de capital
def profile(df: pd.DataFrame):
    profile = pd.DataFrame(
        {
            "dtype": df.dtypes,
            "n_nulls": df.isnull().sum(),
            "n_unique": df.nunique(),
            "min": df.min(numeric_only=True),
            "Q1": df.quantile(0.25, numeric_only=True),
            "median": df.median(numeric_only=True),
            "mean": df.mean(numeric_only=True),
            "Q3": df.quantile(0.75, numeric_only=True),
            "max": df.max(numeric_only=True),
        }
    )
    return profile


profile_df = profile(df)
display(profile_df)

,dtype,n_nulls,n_unique,min,Q1,median,mean,Q3,max
ad_type,object,0,1,NaN,NaN,NaN,NaN,NaN,NaN
bathrooms,float64,37245,18,1.000000,1.000000,1.000000,1.545119,2.000000,2.000000e+01
bedrooms,float64,127386,65,-2.000000,1.000000,2.000000,1.951525,3.000000,3.030000e+02
created_on,datetime64[us],0,346,NaN,NaN,NaN,NaN,NaN,NaN
currency,object,8291,2,NaN,NaN,NaN,NaN,NaN,NaN
description,object,0,156677,NaN,NaN,NaN,NaN,NaN,NaN
end_date,datetime64[us],0,448,NaN,NaN,NaN,NaN,NaN,NaN
id,int64,0,249738,5.000000,216169.000000,463253.500000,469234.024125,693799.000000,1.000000e+06
l1,object,0,1,NaN,NaN,NaN,NaN,NaN,NaN
l2,object,0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# contenido de columnas
for col in ["l1", "l2", "l3", "l4", "l5", "l6"]:
    print(f"\n Valores únicos en {col}:")
    print(df[col].dropna().unique())


 Valores únicos en l1:
['Argentina']

 Valores únicos en l2:
['Capital Federal']

 Valores únicos en l3:
['Colegiales' 'Almagro' 'Villa Urquiza' 'Boedo' 'San Telmo' 'Barrio Norte'
 'Villa Devoto' 'Palermo' 'Monserrat' 'Caballito' 'Balvanera'
 'Villa Ortuzar' 'Villa Crespo' 'Once' 'Saavedra' 'Recoleta'
 'Villa General Mitre' 'Villa del Parque' 'Belgrano' 'Mataderos'
 'Floresta' 'Coghlan' 'Flores' 'Velez Sarsfield' 'Parque Chacabuco'
 'Nuñez' 'Constitución' 'Las Cañitas' 'Villa Riachuelo' 'Puerto Madero'
 'Paternal' 'Congreso' 'Parque Centenario' 'Parque Patricios' 'Pompeya'
 'Villa Lugano' 'San Cristobal' 'Villa Real' 'San Nicolás' 'Boca'
 'Versalles' 'Villa Luro' 'Retiro' 'Chacarita' 'Barracas' 'Monte Castro'
 'Villa Pueyrredón' 'Parque Avellaneda' 'Liniers' 'Abasto' 'Tribunales'
 'Centro / Microcentro' 'Villa Santa Rita' 'Agronomía' 'Parque Chas'
 'Villa Soldati' 'Catalinas' 'Distrito Audiovisual']

 Valores únicos en l4:
['Palermo Viejo' 'Palermo Hollywood' 'Palermo Soho' 'Palermo C

In [9]:
# sacar columnas innecesarias
df = df.drop(
    columns=["title", "description", "ad_type", "id", "l1", "l2", "l4", "l5", "l6"]
)
df.head()

,start_date,end_date,created_on,lat,lon,l3,rooms,bedrooms,bathrooms,surface_total,surface_covered,currency,price_period,property_type,operation_type,price
0,2019-11-29,9999-12-31,2019-11-29,-58.442399,-34.573623,Colegiales,3.0,2.0,2.0,NaN,NaN,USD,None,Departamento,Venta,259000.0
1,2020-06-05,2020-06-08,2020-06-05,-58.430493,-34.606620,Almagro,3.0,2.0,2.0,77.0,67.0,USD,None,Departamento,Venta,235500.0
2,2020-07-01,9999-12-31,2020-07-01,-58.491760,-34.574123,Villa Urquiza,2.0,NaN,1.0,60.0,55.0,USD,None,Departamento,Venta,175000.0
3,2019-08-17,9999-12-31,2019-08-17,-58.420737,-34.631770,Boedo,2.0,1.0,1.0,74.0,47.0,USD,None,PH,Venta,140000.0
4,2019-08-05,2019-08-31,2019-08-05,-58.429983,-34.607225,Almagro,3.0,2.0,1.0,66.0,64.0,USD,None,Departamento,Venta,173000.0


In [10]:
# sacar vacios y duplicados
df_clean = df.dropna().drop_duplicates()

In [11]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249738 entries, 0 to 249737
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   start_date       249738 non-null  datetime64[us]
 1   end_date         249738 non-null  datetime64[us]
 2   created_on       249738 non-null  datetime64[us]
 3   lat              217583 non-null  float64       
 4   lon              217705 non-null  float64       
 5   l3               225456 non-null  object        
 6   rooms            171449 non-null  float64       
 7   bedrooms         122352 non-null  float64       
 8   bathrooms        212493 non-null  float64       
 9   surface_total    171780 non-null  float64       
 10  surface_covered  165345 non-null  float64       
 11  currency         241447 non-null  object        
 12  price_period     99785 non-null   object        
 13  property_type    249738 non-null  object        
 14  operation_type   249

In [12]:
con = duckdb.connect("app/data/DB/entrenamiento.duckdb")

# Registrar el DataFrame como tabla temporal
con.register("df_temp", df)

# guarda datos limpios en la base de datos duckdb
con.execute(
    """
    CREATE TABLE IF NOT EXISTS datos_clean AS 
    SELECT * FROM df_temp
"""
)

In [13]:
tables = con.execute("SHOW TABLES").fetchall()
print(tables)

[('datos_clean',), ('datos_raw',), ('datosraw_',), ('df_temp',)]


In [14]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import TransformedTargetRegressor
import joblib
import os


# --- feature engineering ---
import pandas as pd


def clean_temporal_columns(df, date_cols, placeholder="9999-12-31"):
    """
    Limpia columnas de fecha
    - Reemplaza valores ficticios  '9999-12-31' por pd.NA
    - Convierte a datetime con coerción de errores
    """
    for col in date_cols:
        if col in df.columns:
            # Detecta si el placeholder está presente
            if df[col].astype(str).str.contains(placeholder).any():
                df[col] = df[col].replace(placeholder, pd.NA)
            # Convierte a datetime
            df[col] = pd.to_datetime(df[col], errors="coerce")
    return df


# crear características temporales (days active: end_date - start_date, y  days since created)
# Si end_date es NA, considera days_active como NaN
df["days_active"] = (df["end_date"] - df["start_date"]).dt.days.astype(float)
df["created_age_days"] = (pd.Timestamp.today() - df["created_on"]).dt.days.astype(float)

# eleccion de features
numeric_feats = [
    "lon",
    "lat",
    "rooms",
    "bedrooms",
    "bathrooms",
    "surface_total",
    "surface_covered",
    "days_active",
    "created_age_days",
]
categorical_feats = [
    "l3",
    "currency",
    "price_period",
    "property_type",
    "operation_type",
]

df = df.dropna(subset=["price"])  # saca las filas que tengan NA en price
X = df[numeric_feats + categorical_feats].copy()
y = df["price"].values  # saca el target

In [15]:
# creacion de pipelines

# tarda 4 minutos en ejecutarse con los hiperparametros, si se aumentan, tarda 7 minutos
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- Preprocesamiento para Modelo A---
num_transform_rf = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("robust_scaler", RobustScaler()),
    ]
)

cat_transform = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)


preproc_rf = ColumnTransformer(
    [
        ("num", num_transform_rf, numeric_feats),
        ("cat", cat_transform, categorical_feats),
    ]
)

rf = RandomForestRegressor(
    n_estimators=100,  # 200+ da mejoras marginales, tarda mucho
    max_depth=None,
    min_samples_split=4,
    n_jobs=-1,  # usa todos los núcleos del CPU
    random_state=42,
)


model_rf = Pipeline(
    [
        ("preproc", preproc_rf),
        (
            "est",
            TransformedTargetRegressor(
                regressor=rf, func=np.log1p, inverse_func=np.expm1
            ),
        ),
    ]
)

# --- Preprocesamiento de Model B---
num_transform_gb = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

preproc_gb = ColumnTransformer(
    [
        ("num", num_transform_gb, numeric_feats),
        ("cat", cat_transform, categorical_feats),
    ]
)

gb = GradientBoostingRegressor(n_estimators=200, random_state=42)

model_gb = Pipeline(
    [
        ("preprocessor", preproc_rf),
        (
            "model",
            GradientBoostingRegressor(
                n_estimators=200,
                learning_rate=0.05,
                max_depth=3,
                subsample=0.8,
                random_state=42,
            ),
        ),
    ]
)


# --- entrenamiento ---
model_rf.fit(X_train, y_train)
model_gb.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [16]:
# --- metricas ---
def evaluate(model, X_te, y_te, name="model"):
    y_pred = model.predict(X_te)
    rmse = mean_squared_error(y_te, y_pred) ** 0.5

    mae = mean_absolute_error(y_te, y_pred)
    r2 = r2_score(y_te, y_pred)
    print(f"--- {name} ---")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"R2: {r2:.4f}")
    return {"rmse": float(rmse), "mae": float(mae), "r2": float(r2)}


metrics_rf = evaluate(model_rf, X_test, y_test, name="RandomForest")
metrics_gb = evaluate(model_gb, X_test, y_test, name="GradientBoosting")

# --- guardar con joblib y escribe en manifest.json reemplazar con MLflow si hay tiempo ---
out_dir = "app/data/housing_models"
os.makedirs(out_dir, exist_ok=True)
rf_path = os.path.join(out_dir, "random_forest.joblib")
gb_path = os.path.join(out_dir, "gradient_boosting.joblib")
manifest_path = os.path.join(out_dir, "manifest.json")

joblib.dump(model_rf, rf_path)
joblib.dump(model_gb, gb_path)

manifest = {
    "models": [
        {"name": "RandomForest", "path": rf_path, "metrics": metrics_rf},
        {"name": "GradientBoosting", "path": gb_path, "metrics": metrics_gb},
    ],
    "note": "Modelo guardado con joblib; target used log1p transform",  # no aplicar np.expm1() al predecir
}

with open(manifest_path, "w", encoding="utf-8") as f:
    json.dump(manifest, f, indent=2)

print("\nArchivos guardados:")
print(" -", rf_path)
print(" -", gb_path)
print(" -", manifest_path)

#  diccionario con rutas de archivos guardadas
{"random_forest": rf_path, "gradient_boosting": gb_path, "manifest": manifest_path}

--- RandomForest ---
RMSE: 639861.86
MAE: 53470.56
R2: 0.5335
--- GradientBoosting ---
RMSE: 743255.32
MAE: 123738.60
R2: 0.3706

Archivos guardados:
 - app/data/housing_models\random_forest.joblib
 - app/data/housing_models\gradient_boosting.joblib
 - app/data/housing_models\manifest.json


{'random_forest': 'app/data/housing_models\\random_forest.joblib',
 'gradient_boosting': 'app/data/housing_models\\gradient_boosting.joblib',
 'manifest': 'app/data/housing_models\\manifest.json'}